# very simple example with Measurements mini-ontology

In [126]:
import rdflib

# define an ontology with RDF triples in TTL ("turtle") format
ontology_data = """
prefix : <http://example.org/>
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix unit: <http://qudt.org/vocab/unit/> .

# these classes do nothing, they are for example purposes only
:Measurement a rdfs:Class .
:Length rdfs:subClassOf :Measurement .
:Height rdfs:subClassOf :Measurement .
"""
ontology = rdflib.Graph()
ontology.parse(data=ontology_data, format="turtle")

# define constraint shapes using RDF triples in TTL
shacl_data = """
prefix : <http://example.org/>
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix unit: <http://qudt.org/vocab/unit/> .

:myRule a sh:NodeShape ;             # enforce a Shape on the Node with type Measurement (name <myRule> can be anything)
    sh:targetClass :Measurement ;    # this should apply to :Measurement and all its subClasses
    sh:property [
        sh:path qudt:numericValue ;  # place constraints on the numericValue of the :Measurement
        sh:nodeKind sh:Literal ;     # make sure we are evaluating a literal
        sh:or ( [ sh:datatype xsd:double ] [ sh:datatype xsd:float ] [ sh:datatype xsd:decimal ] ) ;  # enforce certain datatypes
        sh:minInclusive 0.0 ;        # enforce >=0 values
        sh:minCount 1 ;              # enforce at least 1 qudt:numericValue
        sh:maxCount 1 ;              # enforce at most 1 qudt:numericValue
    ] .
"""
shacl = rdflib.Graph()
shacl.parse(data=shacl_data, format="turtle")

<Graph identifier=Nafe6c967461b450d80cbe23f31adf4e3 (<class 'rdflib.graph.Graph'>)>

In [127]:
# define data instances using RDF triples in TTL
data_graph = '''
prefix : <http://example.org/>
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix unit: <http://qudt.org/vocab/unit/> .

# should fail due to negative value, violating minInclusive
:Length1 a :Length ;
  qudt:numericValue -1.0 ;
  qudt:unit unit:CentiM ;
.

# should pass
:Length2 a :Length ;
  qudt:numericValue "2.0"^^xsd:decimal ;
  qudt:unit unit:M ;
.

# should fail due to wrong datatype
:Height1 a :Height ;
  qudt:numericValue "3"^^xsd:str ;
.

# should pass
:Height2 a :Height ;
  qudt:numericValue 4.0 ;
  qudt:unit unit:M ;
.
'''

In [128]:
from pyshacl import validate

# run the validation algorithm
conforms, v_graph, v_text = validate(data_graph, shacl_graph=shacl,
                                     ont_graph=ontology,
                                     data_graph_format="turtle",
                                     shacl_graph_format="turtle",
                                     inference='rdfs', debug=False,
                                     serialize_report_graph=False)
print(v_text)

Validation Report
Conforms: False
Results (2):
Constraint Violation in MinInclusiveConstraintComponent (http://www.w3.org/ns/shacl#MinInclusiveConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:maxCount Literal("1", datatype=xsd:integer) ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:minInclusive Literal("0.0", datatype=xsd:decimal) ; sh:nodeKind sh:Literal ; sh:or ( [ sh:datatype xsd:double ] [ sh:datatype xsd:float ] [ sh:datatype xsd:decimal ] ) ; sh:path qudt:numericValue ]
	Focus Node: :Length1
	Value Node: Literal("-1.0", datatype=xsd:decimal)
	Result Path: qudt:numericValue
	Message: Value is not >= Literal("0.0", datatype=xsd:decimal)
Constraint Violation in OrConstraintComponent (http://www.w3.org/ns/shacl#OrConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:maxCount Literal("1", datatype=xsd:integer) ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:minInclusive Literal("0.0", datatype=xsd:decimal) ; sh:nodeKind sh:Literal ; sh:or ( [ 

In [129]:
# example of SPARQL query to identify the values in violation of SHACL shapes:
sparql_concise = """
SELECT ?focus
  WHERE {
    ?id rdf:type sh:ValidationResult .
    ?id sh:focusNode ?focus .
  }
"""

result = v_graph.query(sparql_concise)
bad_URI = [list(it.values())[0] for it in result.bindings]
bad_nodes = [it.toPython().split('/')[-1] for it in bad_URI]
print(bad_nodes)

['Length1', 'Height1']


# more complex example with shapes and richer metadata

In [58]:
import rdflib

# read an ontology with RDF triples in TTL ("turtle") format
ontology = rdflib.Graph()
ontology.parse('ontology/basic_shapes.ttl', format="turtle")

# read constraint shapes defined using RDF triples in TTL
shacl = rdflib.Graph()
shacl.parse('ontology/shapes_shacl.ttl', format="turtle")

# define a data graph for this instance
data_graph = """
@prefix : <http://sites.psu.edu/reinhartgroup/mykg/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Sample1 a :Cuboid ;
  :measurement :Length1 ;
  :measurement :Height1 ;
  :measurement :Width1 ;
.

:Length1 a :Length ;
  qudt:numericValue 1.0 ;
  qudt:unit unit:CentiM ;
.

:Height1 a :Length ;
  qudt:numericValue -2.0 ;
  qudt:unit unit:CentiM ;
.

:Width1 a :Length ;
  qudt:numericValue 3 ;
  qudt:unit unit:CentiM ;
.
"""

In [59]:
from pyshacl import validate

conforms, v_graph, v_text = validate(data_graph, shacl_graph=shacl,
                                     ont_graph=ontology,
                                     data_graph_format="turtle",
                                     shacl_graph_format="turtle",
                                     inference='rdfs', debug=False,
                                     serialize_report_graph=False)
print(conforms)
print(v_graph)
print(v_text)

False
[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].
Validation Report
Conforms: False
Results (1):
Constraint Violation in MinInclusiveConstraintComponent (http://www.w3.org/ns/shacl#MinInclusiveConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minInclusive Literal("0.0", datatype=xsd:decimal) ; sh:nodeKind sh:Literal ; sh:path qudt:numericValue ]
	Focus Node: :Height1
	Value Node: Literal("-2.0", datatype=xsd:decimal)
	Result Path: qudt:numericValue
	Message: Value is not >= Literal("0.0", datatype=xsd:decimal)



In [108]:
from pyshacl import validate


shapes_file = """
prefix ex: <http://example.org/>
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix unit: <http://qudt.org/vocab/unit/> .

ex:NegativeMeasurementShape a sh:NodeShape ;
    sh:targetClass ex:Measurement ;
    sh:property [
        sh:path qudt:numericValue ;
        sh:nodeKind sh:Literal ;
        sh:or ( [ sh:datatype xsd:double ] [ sh:datatype xsd:float ] [ sh:datatype xsd:decimal ] ) ;
        sh:minInclusive 0.0 ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
    ] ;
    sh:property [
        sh:path qudt:unit ;
        sh:nodeKind sh:IRI ;
        sh:class qudt:Unit ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
    ] .
"""
shapes_file_format = 'turtle'

data_file = '''
prefix : <http://example.org/>
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix unit: <http://qudt.org/vocab/unit/> .

:Length1 a :Length ;
  qudt:numericValue "-1.0"^^xsd:float ;
  qudt:unit unit:CentiM ;
.

:Length2 a :Length ;
  qudt:numericValue "2.0"^^xsd:decimal ;
  qudt:unit unit:M ;
.

:Height1 a :Height ;
  qudt:numericValue "3"^^xsd:double ;
.

:Height2 a :Height ;
  qudt:numericValue 4.0 ;
  qudt:unit :M ;
.
'''
data_file_format = 'turtle'

conforms, v_graph, v_text = validate(data_file, shacl_graph=shapes_file,
                                     ont_graph=ontology,
                                     data_graph_format=data_file_format,
                                     shacl_graph_format=shapes_file_format,
                                     inference='rdfs', debug=False,
                                     serialize_report_graph=False)
print(conforms)
print(v_graph)
print(v_text)

False
[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].
Validation Report
Conforms: False
Results (5):
Constraint Violation in MinInclusiveConstraintComponent (http://www.w3.org/ns/shacl#MinInclusiveConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:maxCount Literal("1", datatype=xsd:integer) ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:minInclusive Literal("0.0", datatype=xsd:decimal) ; sh:nodeKind sh:Literal ; sh:or ( [ sh:datatype xsd:double ] [ sh:datatype xsd:float ] [ sh:datatype xsd:decimal ] ) ; sh:path qudt:numericValue ]
	Focus Node: :Length1
	Value Node: Literal("-1.0", datatype=xsd:float)
	Result Path: qudt:numericValue
	Message: Value is not >= Literal("0.0", datatype=xsd:decimal)
Constraint Violation in ClassConstraintComponent (http://www.w3.org/ns/shacl#ClassConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:class qudt:Unit ; sh:maxCount Literal("1", datatype=xsd:integer) ; sh:minCount Literal("1", datatype=xsd:in

# using kglab

> Note: this doesn't work, I am not presently using `kglab`

In [87]:
from os.path import dirname
import kglab
import os

namespaces = {
    "nom":  "http://example.org/#",
    "wtm":  "http://purl.org/heals/food/",
    "ind":  "http://purl.org/heals/ingredient/",
    "skos": "http://www.w3.org/2004/02/skos/core#",
    }

kg = kglab.KnowledgeGraph(
    name = "A recipe KG example based on Food.com",
    base_uri = "https://www.food.com/recipe/",
    namespaces = namespaces,
    )

kg.load_rdf('ontology/minimal.ttl')
# kg.load_rdf(os.path.join('/Users/wfr5091/Downloads', "recipes.ttl"))

In [ ]:
shape_graph = """
prefix ex: <http://example.org/>
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix unit: <http://qudt.org/vocab/unit/> .

ex:NegativeMeasurementShape a sh:NodeShape ;
    sh:targetClass ex:Measurement ;
    sh:property [
        sh:path qudt:numericValue ;
        sh:nodeKind sh:Literal ;
        sh:or ( [ sh:datatype xsd:double ] [ sh:datatype xsd:float ] [ sh:datatype xsd:decimal ] ) ;
        sh:minInclusive 0.0 ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
    ] ;
    sh:property [
        sh:path qudt:unit ;
        sh:nodeKind sh:IRI ;
        sh:class qudt:Unit ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
    ] .
"""

conforms, report_graph, report_text = kg.validate(
    shacl_graph=shape_graph,
    shacl_graph_format="ttl"
)
print(conforms)
print(report_text)